In [2]:
%reload_ext autoreload
%autoreload 2

import os
import copy 
import shutil
import json

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

import utils as ut
import plot_utils as pu

Get again the orbit of the bodies

In [4]:
initial_conditions_file = '../data/periodic_3b_inits/inits.json'
with open(initial_conditions_file, 'r') as f:
    initial_conditions = json.load(f)

orbit = 'Figure Eight'
inits = initial_conditions[orbit]

T = inits['period']
masses = [inits[str(i)]['mass'] for i in range(3)]
bodies = [ut.Body(inits[str(i)]['mass'],
                  inits[str(i)]['pos'][:2],
                  inits[str(i)]['vel'][:2]) for i in range(3)]

FRAMES = 600
dt = T/FRAMES

sim = ut.NBodySimulation(bodies, e=0.0)
sim.run_simulation(T, dt)
pos = sim.positions

radii = np.array([1, 1, 1])*0.1

99.83%

List files with saved data

In [9]:
N, M = 50, 50
motherfold = f'../results/figure_eight_{N}_{M}/'
save_ind_fold = motherfold + f'indices/'
save_alpha_fold = motherfold + f'time_hit/'

# list files in each folder
files_ind = [save_ind_fold + file for file in os.listdir(save_ind_fold)]
files_alpha = [save_alpha_fold + file for file in os.listdir(save_alpha_fold)]

r = 1.5

In [14]:
figurefold = f'../figures/figure_eight_{N}_{M}/'
savefold = figurefold + 'pngs/'
if not os.path.exists(savefold):
    os.makedirs(savefold)
else:
    # remove all files in folder
    for filename in os.listdir(savefold):
        file_path = os.path.join(savefold, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): 
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

Generate plots for each frame

In [15]:
# Define your colors
c0 = np.array([255, 43, 65]) / 255
c1 = np.array([2, 11, 184]) / 255
c2 = np.array([235, 222, 40]) / 255
colors = [c0, c1, c2]

# Generate figure with custom class in plot_utils.py
Fig = pu.Figure(fig_size=1403, grid=False, theme='default', sw=0, ratio=1)
axes = Fig.axes_flat
fs = Fig.fs
fig = Fig.fig

ax = axes[0]
ax.set_xticks([])
ax.set_yticks([])
ax.margins(x=0, y=0)

background_color = np.array([1, 1, 1])*0

# We create colormaps for each color to give whiter values to gridpoints that hit the bodies faster
cmaps = [LinearSegmentedColormap.from_list("custom_cmap", [background_color, color, 'w']) for color in colors]

# Plot the orbit (1 body)
ax.plot(pos[:, 0, 0], pos[:, 1, 0], c='w', lw=fs*0.2, zorder=1, alpha=0.5)

for i in range(len(files_ind)):

    # load data
    indices = np.load(files_ind[i])
    hit_time = np.load(files_alpha[i])

    # use hit time to create 'alpha values', actually colormap indices
    alpha = copy.deepcopy(hit_time)
    fact = 1

    # Adjust the following parameters to achieve the desired effect
    max_alpha = np.max(alpha)*fact
    alpha = alpha/max_alpha
    alpha = alpha**(1)
    alpha=1-alpha
    alpha = np.clip(alpha, 0.4, 0.9)

    # generate the rgba image
    rgba_image = np.zeros((*indices.shape, 4))
    # Apply the colormaps
    for j, cmap in enumerate(cmaps):
        mask = indices == j
        rgba_image[mask] = cmap(alpha[mask])

    # Handle the case where indices value is -1 (particle did not hit body)
    mask = indices == -1
    rgba_image[mask] = list(background_color) + [1] 


    im = ax.imshow(rgba_image, extent=[-r, r, -r, r], zorder=0)

    circles = []
    for j in range(len(radii)):
        circle = patches.Circle((pos[i, 0, j], pos[i, 1, j]), radii[j], 
                                facecolor=colors[j], 
                                edgecolor='w',
                                linewidth = fs*0,
                                fill=True,
                                zorder=2)
        ax.add_patch(circle)
        circles.append(circle)

    filename = f'render_{i:04d}.jpg'
    Fig.save(savefold+filename, bbox_inches='tight', pad_inches=0)

    for circle in circles:
        circle.remove()
    
    im.remove()

    plt.close()

Generate video

In [13]:
pu.png_to_mp4(savefold, fps=60, title='video')